In [ ]:
# -*- coding: utf-8 -*-
"""
Batch fetch abstracts for Kellogg papers (resumable version)
Pipeline: Crossref → Semantic Scholar (incl. DOI direct lookup) → Google Scholar (SerpAPI as fallback)

Input file: YOUR_FILE_NAME.xlsx
Required columns: title / authors / year / url / abstract (abstract can be empty)

- Automatic resume-point: based on KEY (sha1 of title + authors + year + url)
- Periodic autosave: *_autosave_with_abstracts.xlsx / *_autosave_need_manual.xlsx
- 429/network errors: exponential backoff + respect Retry-After
"""

# ======【Config】======
INPUT_FILE        = "YOUR_FILE_NAME.xlsx"   # Excel file for Kellogg papers
PROFESSOR_NAME    = None           # Filter by professor name substring; otherwise None
S2_API_KEY        = None           # Optional: Semantic Scholar API key (not required)
SERPAPI_KEY       = "YOUR API KEY"  # SerpAPI Key

# Runtime parameters
REQUEST_INTERVAL  = 1.0            # Minimum delay per request (be polite)
TOP_K             = 3              # Candidate count for each source
FUZZ_THRESHOLD    = 78             # change as you needed
AUTOSAVE_EVERY    = 50             # Autosave every N rows

import os, re, time, html, sys, json, math, hashlib, requests, pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
from rapidfuzz import fuzz

# Rough pattern for identifying likely "no public abstract" works
LIKELY_NO_ABS_PAT = re.compile(
    r"(forthcoming|in\s*press|pre[-\s]*print|working\s*paper|under\s*review|submitted|ssrn|arxiv|nber)",
    re.I
)

# ========== Helpers ==========
def row_key(title: str, authors: str, year, url: str = "") -> str:
    """
    Generate KEY from Kellogg columns, used for resumable dedup:
    sha1(title + authors + year + url)
    """
    base = f"{str(title).strip()}||{str(authors).strip()}||{str(year).strip()}||{str(url).strip()}"
    return hashlib.sha1(base.encode("utf-8")).hexdigest()

def jats_to_text(jats: str) -> str:
    s = html.unescape(jats or "")
    s = re.sub(r"<[^>]+>", " ", s)
    return re.sub(r"\s+", " ", s).strip()

def http_get(url, params=None, timeout=25, headers=None, tries=6, sleep=0.8):
    """Generic GET: handle 429 (respect Retry-After), 5xx (exponential backoff), retry on limited attempts."""
    last_status = None
    for k in range(tries):
        try:
            r = requests.get(url, params=params, timeout=timeout, headers=headers)
            if r.status_code == 200:
                return r
            if r.status_code == 429:
                ra = r.headers.get("Retry-After")
                wait = float(ra) if ra else min(sleep * (2 ** k), 64.0)
                time.sleep(wait)
                last_status = 429
                continue
            if 500 <= r.status_code < 600:
                time.sleep(min(sleep * (k + 1), 30.0))
                last_status = r.status_code
                continue
            r.raise_for_status()
        except requests.RequestException:
            time.sleep(min(sleep * (k + 1), 30.0))
            last_status = "network"
    raise RuntimeError(f"GET fail: {url} | status={last_status}")

def safe_to_excel(df: pd.DataFrame, path: str):
    for _ in range(3):
        try:
            df.to_excel(path, index=False)
            return
        except Exception:
            time.sleep(1.0)

# ========== Data Sources ==========
def search_crossref_biblio(q, rows=TOP_K):
    url = "https://api.crossref.org/works"
    params = {
        "query.bibliographic": q,
        "rows": rows,
        "select": "title,abstract,issued,container-title,DOI,URL"
    }
    r = http_get(url, params=params, headers={"User-Agent": "abstract-fetcher/1.0"})
    return (r.json().get("message") or {}).get("items", [])

# —— Semantic Scholar ——
def s2_headers():
    h = {"User-Agent": "abstract-fetcher/1.0"}
    if S2_API_KEY:
        h["x-api-key"] = S2_API_KEY
    return h

def search_semantic_scholar(q, rows=TOP_K):
    """
    Search by title/authors.
    fields: title, abstract, year, venue, url, externalIds
    """
    url = "https://api.semanticscholar.org/graph/v1/paper/search"
    params = {
        "query": q,
        "limit": rows,
        "fields": "title,abstract,year,venue,url,externalIds"
    }
    r = http_get(url, params=params, headers=s2_headers(), timeout=30, tries=5, sleep=1.0)
    return (r.json() or {}).get("data", []) or []

def fetch_semantic_by_doi(doi: str):
    """Semantic Scholar: direct fetch via DOI"""
    if not doi:
        return None
    url = f"https://api.semanticscholar.org/graph/v1/paper/DOI:{doi}"
    params = {"fields": "title,abstract,year,venue,url,externalIds"}
    try:
        r = http_get(url, params=params, headers=s2_headers(), timeout=30, tries=5, sleep=1.0)
        return r.json()
    except Exception:
        return None

# —— Google Scholar (SerpAPI fallback, using snippet as weak abstract) ——
def search_google_scholar_serpapi(title: str, author_hint: str = ""):
    """Return raw SerpAPI results; best title match used as weak abstract"""
    if not SERPAPI_KEY or SERPAPI_KEY.startswith("PUT_"):
        return None
    url = "https://serpapi.com/search.json"
    q = f'{title} {author_hint}'.strip()
    params = {
        "engine": "google_scholar",
        "q": q,
        "api_key": SERPAPI_KEY,
        "hl": "en",
        "num": 10
    }
    try:
        r = http_get(url, params=params, timeout=30, headers={"User-Agent": "abstract-fetcher/1.0"}, tries=4, sleep=1.2)
        data = r.json()
        items = data.get("organic_results", []) or []
        return items
    except Exception:
        return None

def best_by_fuzz(target, items, key):
    best, score = None, 0
    for it in items:
        t = key(it) or ""
        s = fuzz.token_set_ratio(target, t)
        if s > score:
            best, score = it, s
    return (best, score) if score >= FUZZ_THRESHOLD else (None, score)

# ========== Fetch One ==========
def fetch_one(title_guess: str, authors: str, year):
    """
    Only rely on Kellogg columns: title_guess / authors / year.
    No dependency on any 'Entry' parsing.
    """
    debug = {
        "QueryUsed": "",
        "CandidateTitle": "",
        "CandidateDOI": "",
        "WhyNoAbstract": ""
    }

    # Rough guess about lacking public abstract
    meta_text = f"{title_guess} {authors}"
    likely_no_abs = bool(LIKELY_NO_ABS_PAT.search(meta_text or ""))

    # 1) Crossref
    q_parts = [title_guess]
    if authors:
        q_parts.append(authors)
    if year:
        q_parts.append(str(year))
    q = " ".join(q_parts).strip()

    items = search_crossref_biblio(q)
    debug["QueryUsed"] = f"CR-biblio:{q[:120]}"

    # Sort by closeness of year
    if year:
        def year_from_item(x):
            issued = x.get("issued", {}).get("date-parts") or [[None]]
            return issued[0][0] or 9999
        items = sorted(items, key=lambda x: abs(year_from_item(x) - int(year)))

    cand, score = best_by_fuzz(
        title_guess,
        items,
        key=lambda x: (x.get("title") or [""])[0] if x.get("title") else ""
    )

    s2_after_cr = None

    if cand:
        ctitle = (cand.get("title") or [""])[0] if cand.get("title") else ""
        doi    = cand.get("DOI", "")
        abs_cr = jats_to_text(cand.get("abstract", "") or "")
        debug.update({"CandidateTitle": ctitle, "CandidateDOI": doi})

        if abs_cr:
            res = {
                "Abstract": abs_cr,
                "DOI": doi,
                "URL": cand.get("URL", ""),
                "Year_Found": (cand.get("issued", {}).get("date-parts") or [[None]])[0][0],
                "Venue": (cand.get("container-title") or [""])[0] if cand.get("container-title") else "",
                "Source": "Crossref",
                "Status": "ok",
            }
            res.update(debug)
            return res, score

        # No abstract: try Semantic Scholar via DOI
        s2_after_cr = {
            "Abstract": "",
            "DOI": doi,
            "URL": cand.get("URL", ""),
            "Year_Found": (cand.get("issued", {}).get("date-parts") or [[None]])[0][0],
            "Venue": (cand.get("container-title") or [""])[0] if cand.get("container-title") else "",
            "Source": "Crossref",
            "Status": "no_abstract_yet" if likely_no_abs else "no_abstract_from_cr",
        }
        s2_after_cr.update(debug)
        try:
            if doi:
                s2 = fetch_semantic_by_doi(doi)
                if s2 and (s2.get("abstract") or ""):
                    s2_abs = (s2.get("abstract") or "").strip()
                    venue  = s2.get("venue", "") or s2_after_cr.get("Venue", "")
                    url    = s2.get("url", "") or s2_after_cr.get("URL", "")
                    year_f = s2.get("year", "") or s2_after_cr.get("Year_Found", "")
                    s2_after_cr.update({
                        "Abstract": s2_abs,
                        "Venue": venue,
                        "URL": url,
                        "Year_Found": year_f,
                        "Source": "Semantic Scholar (by DOI)",
                        "Status": "ok",
                    })
                    return s2_after_cr, score
        except Exception:
            pass

    time.sleep(REQUEST_INTERVAL)

    # 2) Semantic Scholar (title+authors)
    try:
        s2_list = search_semantic_scholar(f"{title_guess} {authors}".strip())
    except Exception:
        s2_list = []

    cand_s2, score_s2 = None, 0
    for it in s2_list:
        t = it.get("title", "") or ""
        s = fuzz.token_set_ratio(title_guess, t)
        if s > score_s2:
            cand_s2, score_s2 = it, s

    if cand_s2:
        abs_txt = (cand_s2.get("abstract") or "").strip()
        if abs_txt:
            ext        = cand_s2.get("externalIds") or {}
            doi        = ext.get("DOI", "") or ""
            url        = cand_s2.get("url", "") or ""
            venue      = (cand_s2.get("venue") or "") or ""
            year_found = cand_s2.get("year", "") or ""
            res = {
                "Abstract": abs_txt,
                "DOI": doi,
                "URL": url,
                "Year_Found": year_found,
                "Venue": venue,
                "Source": "Semantic Scholar",
                "Status": "ok",
            }
            res.update(debug)
            return res, score_s2

    time.sleep(REQUEST_INTERVAL)

    # 3) Google Scholar (SerpAPI fallback)
    try:
        gs = search_google_scholar_serpapi(title_guess, author_hint=authors)
        if gs:
            cand4, score4 = best_by_fuzz(title_guess, gs, key=lambda x: x.get("title", ""))
            if cand4 and score4 >= FUZZ_THRESHOLD:
                snippet = cand4.get("snippet", "") or ""
                pub_info = cand4.get("publication_info", {}) or {}
                year_found = None
                if isinstance(pub_info, dict):
                    yf = re.search(r"\b(20\d{2}|19\d{2})\b", json.dumps(pub_info))
                    year_found = int(yf.group(1)) if yf else ""
                res = {
                    "Abstract": snippet,
                    "DOI": "",
                    "URL": cand4.get("link", "") or "",
                    "Year_Found": year_found or "",
                    "Venue": pub_info.get("summary", "") if isinstance(pub_info, dict) else "",
                    "Source": "Google Scholar (SerpAPI)",
                    "Status": "ok-snippet",
                }
                res.update(debug)
                return res, score4
    except Exception as e:
        debug["WhyNoAbstract"] += f"GS-fail({type(e).__name__}); "

    # 4) No abstract found in all sources
    final = s2_after_cr or {
        "Abstract": "",
        "DOI": "",
        "URL": "",
        "Year_Found": "",
        "Venue": "",
        "Source": ""
    }
    why = "No abstract in CR/S2/GS"
    if likely_no_abs:
        why = "Likely no public abstract (forthcoming/pre/working)"
        final["Status"] = "likely_no_public_abstract"
    else:
        final["Status"] = final.get("Status", "no_abstract_all_sources")
    final.update({"WhyNoAbstract": why})
    final.update(debug)
    return final, 0

# ========== Resume-point Loader ==========
def load_done_keys(prefix: str):
    done = set()
    for suf in [
        "_with_abstracts.xlsx",
        "_need_manual.xlsx",
        "_autosave_with_abstracts.xlsx",
        "_autosave_need_manual.xlsx"
    ]:
        p = Path(f"{prefix}{suf}")
        if p.exists():
            try:
                df = pd.read_excel(p)
                if "KEY" in df.columns:
                    done.update(df["KEY"].dropna().astype(str).tolist())
                else:
                    df = df.copy()
                    df["_KEY"] = df.apply(
                        lambda r: row_key(
                            r.get("title", ""),
                            r.get("authors", ""),
                            r.get("year", ""),
                            r.get("url", "")
                        ),
                        axis=1
                    )
                    done.update(df["_KEY"].tolist())
            except Exception:
                pass
    return done

# ========== Batch main workflow ==========
def run_batch(df: pd.DataFrame, out_prefix: str):
    # Generate KEY
    if "KEY" not in df.columns:
        df["KEY"] = df.apply(
            lambda r: row_key(
                r.get("title", ""),
                r.get("authors", ""),
                r.get("year", ""),
                r.get("url", "")
            ),
            axis=1
        )

    # Resume: load completed keys
    already = load_done_keys(out_prefix)

    # Only process rows with empty abstract and not in completed keys
    mask_no_abs = df["abstract"].isna() | (df["abstract"].astype(str).str.strip() == "")
    todo = df[mask_no_abs & (~df["KEY"].isin(already))].copy()

    rows, seen = [], 0
    pbar = tqdm(total=len(todo), desc="🔎 Fetching abstracts (resumable)")

    for _, r in todo.iterrows():
        title_guess = str(r.get("title", "") or "").strip()
        authors     = str(r.get("authors", "") or "").strip()
        year        = r.get("year", "")

        try:
            res, score = fetch_one(title_guess, authors, year)
        except Exception as e:
            res, score = ({"Abstract": "", "Source": "", "Status": f"error:{type(e).__name__}"}, 0)

        rec = {
            "KEY": r.get("KEY", ""),
            "title": title_guess,
            "authors": authors,
            "year": year,
            "url": r.get("url", ""),
            "Abstract": res.get("Abstract", ""),
            "Source": res.get("Source", ""),
            "MatchScore": score,
            "DOI": res.get("DOI", ""),
            "URL_found": res.get("URL", ""),
            "Year_Found": res.get("Year_Found", ""),
            "Venue": res.get("Venue", ""),
            "Status": res.get("Status", ""),
            "QueryUsed": res.get("QueryUsed", ""),
            "CandidateTitle": res.get("CandidateTitle", ""),
            "CandidateDOI": res.get("CandidateDOI", ""),
            "WhyNoAbstract": res.get("WhyNoAbstract", ""),
        }
        rows.append(rec)

        seen += 1
        pbar.update(1)

        # Periodic autosave
        if seen % AUTOSAVE_EVERY == 0:
            tmp = pd.DataFrame(rows)
            out_ok   = tmp[tmp["Abstract"].astype(str).str.len() > 0]
            out_todo = tmp[tmp["Abstract"].astype(str).str.len() == 0]
            if len(out_ok):
                safe_to_excel(out_ok,   f"{out_prefix}_autosave_with_abstracts.xlsx")
            if len(out_todo):
                safe_to_excel(out_todo, f"{out_prefix}_autosave_need_manual.xlsx")

        time.sleep(REQUEST_INTERVAL)

    pbar.close()

    cur = pd.DataFrame(rows)

    def load_old(suf):
        p = Path(f"{out_prefix}{suf}")
        if p.exists():
            try:
                return pd.read_excel(p)
            except Exception:
                return pd.DataFrame()
        return pd.DataFrame()

    old_with  = load_old("_with_abstracts.xlsx")
    old_need  = load_old("_need_manual.xlsx")
    auto_with = load_old("_autosave_with_abstracts.xlsx")
    auto_need = load_old("_autosave_need_manual.xlsx")

    all_old = pd.concat([old_with, old_need, auto_with, auto_need], ignore_index=True, sort=False)

    if len(all_old):
        if "KEY" not in all_old.columns:
            all_old["KEY"] = all_old.apply(
                lambda r: row_key(
                    r.get("title", ""),
                    r.get("authors", ""),
                    r.get("year", ""),
                    r.get("url", "")
                ),
                axis=1
            )
        merged = pd.concat([all_old, cur], ignore_index=True, sort=False)
        merged = merged.drop_duplicates(subset=["KEY"], keep="last")
    else:
        merged = cur

    out_ok   = merged[merged["Abstract"].astype(str).str.len() > 0]
    out_todo = merged[merged["Abstract"].astype(str).str.len() == 0]

    safe_to_excel(out_ok,   f"{out_prefix}_with_abstracts.xlsx")
    safe_to_excel(out_todo, f"{out_prefix}_need_manual.xlsx")

    print(f"Done. With abstracts: {len(out_ok)} | Need manual: {len(out_todo)} | Total: {len(merged)}")

# ========== Entry ==========
if __name__ == "__main__":
    df = pd.read_excel(INPUT_FILE)

    # Optional: filter by professor name
    if PROFESSOR_NAME:
        df = df[df["authors"].astype(str).str.contains(PROFESSOR_NAME, case=False, na=False)].copy()
        if df.empty:
            print(f"[⚠] Professor not found: {PROFESSOR_NAME}")
            sys.exit(0)
        out_prefix = re.sub(r"[^A-Za-z0-9]+", "_", PROFESSOR_NAME).strip("_")
    else:
        out_prefix = Path(INPUT_FILE).stem

    # Generate KEY
    if "KEY" not in df.columns:
        df["KEY"] = df.apply(
            lambda r: row_key(
                r.get("title", ""),
                r.get("authors", ""),
                r.get("year", ""),
                r.get("url", "")
            ),
            axis=1
        )

    run_batch(df, out_prefix)
